In [0]:
%sql

with preprocessed_cast(

  select cast_name,
  primary_profession,
  known_for_title,
  row_number() over (partition by cast_name order by primary_profession) as r_no
  from ott_catalog.silver.dim_cast
)
merge into ott_catalog.gold.dim_cast t
using (

select cast_name,
primary_profession,
known_for_title
from preprocessed_cast
where r_no=1

) s
on t.cast_name=s.cast_name
when matched then
update set t.primary_profession=s.primary_profession,
t.known_for_title=s.known_for_title,
t.updated_at=current_timestamp()
when not matched then 
insert(
  cast_name,
  primary_profession,
  known_for_title,
  created_at,
  updated_at
)
values(
  s.cast_name,
  s.primary_profession,
  s.known_for_title,
  current_timestamp(),
  current_timestamp()
)

In [0]:
%sql
merge into ott_catalog.gold.dim_category t
using ott_catalog.silver.dim_category s
on t.category_name=s.category_name
when matched then update
set t.category_type=s.category_type,t.updated_at=current_timestamp()
when not matched then insert(
  category_name,
  category_type,
  created_at,
  updated_at
)
values(
  s.category_name,
  s.category_type,
  current_timestamp(),
  current_timestamp()
)

In [0]:
%sql
merge into ott_catalog.gold.dim_countries t
using ott_catalog.silver.dim_countries s
on t.country_name=s.country_name
when matched then update
set t.continent=s.continent,t.market_type=s.market_type,t.updated_at=current_timestamp()
when not matched then insert(
  country_name,
  continent,
  market_type,
  created_at,
  updated_at
)
values(
  s.country_name,
  s.continent,
  s.market_type,
  current_timestamp(),
  current_timestamp()
)

In [0]:
%sql
merge into ott_catalog.gold.dim_directors t
using ott_catalog.silver.dim_directors s
on t.director_name=s.director_name
when matched then update
set t.directing_style=s.directing_style,t.active_years=s.active_years,t.updated_at=current_timestamp()
when not matched then insert(
  director_name,
  directing_style,
  active_years,
  created_at,
  updated_at
)
values(
  s.director_name,
  s.directing_style,
  s.active_years,
  current_timestamp(),
  current_timestamp()
)

In [0]:
%sql
merge into ott_catalog.gold.dim_titles  t
using ott_catalog.silver.dim_titles s
on t.show_id=s.show_id
when matched then update
set t.title_name=s.title_name,t.duration_minutes=s.duration_minutes,t.duration_seasons=s.duration_seasons,
t.type=s.type,t.date_added=s.date_added,t.release_year=s.release_year,t.rating=s.rating,t.description=s.description,
t.updated_at=current_timestamp()
when not matched then insert(
 title_name,
 duration_minutes,
 duration_seasons,
 type,
 date_added,
 release_year,
 rating,
 description,
 show_id,
 created_at,
 updated_at
)
values(
 s.title_name,
 s.duration_minutes,
 s.duration_seasons,
 s.type,
 s.date_added,
 s.release_year,
 s.rating,
 s.description,
 s.show_id,
 current_timestamp(),
 current_timestamp()
)

##Facts-Load-Silver-To-Gold

In [0]:
%sql
with preprocessed_facts as (

select t.show_key,
ts.snapshot_date,
ts.duration_minutes,
ts.duration_seasons,
ts.is_movie_flag,
ts.is_series_flag,
ts.is_new_release_flag
from ott_catalog.gold.dim_titles t
inner join ott_catalog.silver.fact_title_snapshot ts 
on t.show_id=ts.show_id

)

merge into ott_catalog.gold.fact_title_snapshot t
using preprocessed_facts s
on t.show_key=s.show_key and t.snapshot_date=s.snapshot_date
when matched then update
set t.duration_minutes=s.duration_minutes,t.duration_seasons=s.duration_seasons,t.is_movie_flag=s.is_movie_flag,
t.is_new_release_flag=s.is_new_release_flag,t.is_series_flag=s.is_series_flag,t.updated_at=current_timestamp()
when not matched then insert(
 show_key,
 snapshot_date,
 duration_minutes,
 duration_seasons,
 is_movie_flag,
 is_series_flag,
 is_new_release_flag,
 created_at,
 updated_at
)
values(
 s.show_key,
 s.snapshot_date,
 s.duration_minutes,
 s.duration_seasons,
 s.is_movie_flag,
 s.is_series_flag,
 s.is_new_release_flag,
 current_timestamp(),
 current_timestamp()
)


##Bridge Tables -Load-Silver-To-Gold

In [0]:
%sql
with preprocessed_bridge(

    select distinct
          s.show_key,
          c.cast_key

    from ott_catalog.silver.show_cast_rel sc
    inner join ott_catalog.gold.dim_titles s on sc.show_id=s.show_id
    inner join ott_catalog.gold.dim_cast c on sc.cast_name=c.cast_name
)

merge into ott_catalog.gold.bridge_show_cast t
using preprocessed_bridge s
on t.show_key=s.show_key and t.cast_key=s.cast_key
when not matched then insert (
  show_key,
  cast_key
)
values(
  s.show_key,
  s.cast_key
)

In [0]:
%sql
with preprocessed_bridge(

    select distinct
          s.show_key,
          c.category_key

    from ott_catalog.silver.show_category_rel sc
    inner join ott_catalog.gold.dim_titles s on sc.show_id=s.show_id
    inner join ott_catalog.gold.dim_category c on sc.category_name=c.category_name
)

merge into ott_catalog.gold.bridge_show_category t
using preprocessed_bridge s
on t.show_key=s.show_key and t.category_key=s.category_key
when not matched then insert (
  show_key,
  category_key
)
values(
  s.show_key,
  s.category_key
)

In [0]:
%sql
with preprocessed_bridge(

    select distinct
          s.show_key,
          c.country_key

    from ott_catalog.silver.show_countries_rel sc
    inner join ott_catalog.gold.dim_titles s on sc.show_id=s.show_id
    inner join ott_catalog.gold.dim_countries c on sc.country_name=c.country_name
)

merge into ott_catalog.gold.bridge_show_countries t
using preprocessed_bridge s
on t.show_key=s.show_key and t.country_key=s.country_key
when not matched then insert (
  show_key,
  country_key
)
values(
  s.show_key,
  s.country_key
)

In [0]:
%sql
with preprocessed_bridge(

    select distinct
          s.show_key,
          c.director_key

    from ott_catalog.silver.show_directors_rel sc
    inner join ott_catalog.gold.dim_titles s on sc.show_id=s.show_id
    inner join ott_catalog.gold.dim_directors c on sc.director_name=c.director_name
)

merge into ott_catalog.gold.bridge_show_directors t
using preprocessed_bridge s
on t.show_key=s.show_key and t.director_key=s.director_key
when not matched then insert (
  show_key,
  director_key
)
values(
  s.show_key,
  s.director_key
)